In [1]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

In [2]:
# Configuração
base_path = "../data/processed/"
os.makedirs("../models", exist_ok=True)

# 1. Carregar Dados
df = pd.read_csv(f"{base_path}df_modelagem_v3.csv", index_col="data", parse_dates=True)

# --- DEFINIÇÃO MANUAL DAS FEATURES ---
# Vamos usar APENAS o que queremos simular. Removemos todo o resto.
# Não usaremos 'target_lag_1' para evitar que o modelo fique "preguiçoso".
features_focadas = [
    'selic_lag_6',  # Nossa alavanca principal
    'ipca_lag_6',   # Contexto inflacionário
    'dolar_ptax_lag_6', # Importante para Rural
    'mes',          # Sazonalidade
    'periodo_safra' # Sazonalidade Rural
]

targets = {
    "PF": "inad_pf_tot",
    "PJ": "inad_pj_tot",
    "Rural_PF": "inad_rd_pf_cr_rur_tot", 
    "Rural_PJ": "inad_rd_pj_cr_rur_tot"   
}

print(f"🎯 Iniciando Treinamento Focado. Features: {features_focadas}")

for nome_modelo, target_col in targets.items():
    if target_col not in df.columns: continue
    
    print(f"--- Treinando: {nome_modelo} ---")
    
    # Filtrar Dataset
    # Se alguma coluna focada não existir (ex: dolar), o script avisa, mas vamos assumir que criou no passo 3
    cols_existentes = [c for c in features_focadas if c in df.columns]
    
    df_temp = df.copy().dropna()
    
    X = df_temp[cols_existentes]
    y = df_temp[target_col] # Previsão do valor absoluto direto (Regressão aguenta)
    
    # Padronização
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Modelo Linear (Ridge)
    # Alpha menor aqui para deixar o modelo mais "livre" para obedecer a Selic
    model = Ridge(alpha=0.5) 
    model.fit(X_scaled, y)
    
    # DIAGNÓSTICO DE SENSIBILIDADE (Vamos ver o peso agora!)
    coefs = pd.DataFrame({'var': X.columns, 'coef': model.coef_})
    selic_peso = coefs.loc[coefs['var'] == 'selic_lag_6', 'coef'].sum()
    print(f"   > Peso da Selic: {selic_peso:.4f}")
    
    if abs(selic_peso) < 0.01:
        print("   ⚠️ AVISO: A Selic está fraca. O gráfico mal vai mexer.")
    else:
        print("   ✅ SUCESSO: A Selic tem impacto relevante!")

    # Salvar
    joblib.dump(model, f"../models/model_{nome_modelo}.pkl")
    joblib.dump(scaler, f"../models/scaler_{nome_modelo}.pkl")
    # Salvamos a lista exata de colunas para o app não se perder
    pd.DataFrame(columns=X.columns).to_csv(f"../models/columns_{nome_modelo}.csv", index=False)
    # Valores iniciais
    X.iloc[-1].to_csv(f"../models/last_values_{nome_modelo}.csv")

print("\n🚀 Treino concluído! Este modelo é obrigado a usar a Selic.")

🎯 Iniciando Treinamento Focado. Features: ['selic_lag_6', 'ipca_lag_6', 'dolar_ptax_lag_6', 'mes', 'periodo_safra']
--- Treinando: PF ---
   > Peso da Selic: 0.3084
   ✅ SUCESSO: A Selic tem impacto relevante!
--- Treinando: PJ ---
   > Peso da Selic: 0.3819
   ✅ SUCESSO: A Selic tem impacto relevante!
--- Treinando: Rural_PF ---
   > Peso da Selic: -0.1000
   ✅ SUCESSO: A Selic tem impacto relevante!
--- Treinando: Rural_PJ ---
   > Peso da Selic: -0.8430
   ✅ SUCESSO: A Selic tem impacto relevante!

🚀 Treino concluído! Este modelo é obrigado a usar a Selic.
